# Board Game Recommender
## Collaborative filtering based on game description

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

#ML Packages

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV

# Step 1: Clean and Prepare Data

## EDA


### Explore the games data, then clean and transform into a useable format

In [45]:
# Import games data
df = pd.read_csv('games.csv')

df_games = df.copy(deep=True)

df_games.head(50)

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
0,1,Die Macher,die macher game seven sequential political rac...,1986,4.3206,7.61428,7.10363,1.57979,3,5,...,21926,21926,0,1,0,0,0,0,0,0
1,2,Dragonmaster,dragonmaster tricktaking card game base old ga...,1981,1.9630,6.64537,5.78447,1.45440,3,4,...,21926,21926,0,1,0,0,0,0,0,0
2,3,Samurai,samurai set medieval japan player compete gain...,1998,2.4859,7.45601,7.23994,1.18227,2,4,...,21926,21926,0,1,0,0,0,0,0,0
3,4,Tal der Könige,triangular box luxurious large block tal der k...,1992,2.6667,6.60006,5.67954,1.23129,2,4,...,21926,21926,0,0,0,0,0,0,0,0
4,5,Acquire,acquire player strategically invest business t...,1964,2.5031,7.33861,7.14189,1.33583,2,6,...,21926,21926,0,1,0,0,0,0,0,0
5,6,Mare Mediterraneum,ancient land mediterranean player attempt sati...,1989,3.0000,6.55370,5.54614,1.65350,2,6,...,21926,21926,0,0,0,0,0,0,0,0
6,7,Cathedral,cathedral player set piece different color pie...,1978,1.7950,6.52083,6.13713,1.32963,2,2,...,21926,21926,0,0,0,0,0,1,0,0
7,8,Lords of Creation,interesting offering warfrog player god seek d...,1993,2.4000,6.10716,5.56602,1.32872,2,5,...,21926,21926,0,0,0,0,0,0,0,0
8,9,El Caballero,refer sequel el grande el caballero share aspe...,1998,3.1824,6.45265,5.92290,1.43335,2,4,...,21926,21926,0,1,0,0,0,0,0,0
9,10,Elfenland,elfenland redesign original white wind game el...,1998,2.1578,6.69695,6.47733,1.25365,2,6,...,21926,21926,0,0,0,1,0,0,0,0


In [3]:
#get description of features in df

df_games.info()
df_describe = df_games.describe()

with pd.option_context('display.max_columns', None):
  display(df_describe)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21925 entries, 0 to 21924
Data columns (total 48 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   BGGId                21925 non-null  int64  
 1   Name                 21925 non-null  object 
 2   Description          21924 non-null  object 
 3   YearPublished        21925 non-null  int64  
 4   GameWeight           21925 non-null  float64
 5   AvgRating            21925 non-null  float64
 6   BayesAvgRating       21925 non-null  float64
 7   StdDev               21925 non-null  float64
 8   MinPlayers           21925 non-null  int64  
 9   MaxPlayers           21925 non-null  int64  
 10  ComAgeRec            16395 non-null  float64
 11  LanguageEase         16034 non-null  float64
 12  BestPlayers          21925 non-null  int64  
 13  GoodPlayers          21925 non-null  object 
 14  NumOwned             21925 non-null  int64  
 15  NumWant              21925 non-null 

,BGGId,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,ComAgeRec,LanguageEase,BestPlayers,NumOwned,NumWant,NumWish,NumWeightVotes,MfgPlaytime,ComMinPlaytime,ComMaxPlaytime,MfgAgeRec,NumUserRatings,NumComments,NumAlternates,NumExpansions,NumImplementations,IsReimplementation,Kickstarted,Rank:boardgame,Rank:strategygames,Rank:abstracts,Rank:familygames,Rank:thematic,Rank:cgs,Rank:wargames,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
count,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,16395.000000,16034.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.0,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.00000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000,21925.000000
mean,117652.663216,1985.494914,1.982131,6.424922,5.685673,1.516374,2.007343,5.707868,10.004391,216.461819,0.311517,1467.848164,41.690946,228.457013,49.480137,90.513523,63.678586,90.513523,9.613409,861.668324,0.0,1.603786,1.380068,0.308734,0.116762,0.153341,11019.514071,19729.826956,20839.347184,19732.270011,20736.185815,21625.086796,18680.185678,21295.352201,21062.680274,0.055827,0.10577,0.161003,0.105633,0.013820,0.050855,0.029190,0.040182
std,104628.721777,212.486214,0.848983,0.932477,0.365311,0.285578,0.693093,15.014643,3.269157,236.595136,1.067002,5294.120574,117.255229,788.477151,205.762375,529.657389,443.916212,529.657389,3.641560,3638.680857,0.0,9.619364,7.701036,0.848095,0.321143,0.360324,6372.926817,6389.614077,4695.170448,6387.089598,4893.928466,2542.033794,7420.906104,3637.139987,4219.776597,0.229592,0.30755,0.367542,0.307374,0.116745,0.219707,0.168344,0.196391
min,1.000000,-3500.000000,0.000000,1.041330,3.574810,0.196023,0.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12346.000000,2001.000000,1.333300,5.836960,5.510300,1.320720,2.000000,4.000000,8.000000,24.027778,0.000000,150.000000,3.000000,14.000000,4.000000,25.000000,20.000000,25.000000,8.000000,56.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,5488.000000,21926.000000,21926.000000,21926.000000,21926.000000,21926.000000,21926.000000,21926.000000,21926.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,105305.000000,2011.000000,1.968800,6.453950,5.546540,1.476880,2.000000,4.000000,10.000000,138.000000,0.000000,320.000000,9.000000,39.000000,9.000000,45.000000,30.000000,45.000000,10.000000,123.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,11022.000000,21926.000000,21926.000000,21926.000000,21926.000000,21926.000000,21926.000000,21926.000000,21926.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,206169.000000,2017.000000,2.525200,7.052450,5.679890,1.665470,2.000000,6.000000,12.000000,351.000000,0.000000,899.000000,28.000000,127.000000,26.000000,90.000000,60.000000,90.000000,12.000000,395.000000,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,16544.000000,21926.000000,21926.000000,21926.000000,21926.000000,21926.000000,21926.000000,21926.000000,21926.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,349161.000000,2021.000000,5.000000,9.914290,8.514880,4.277280,10.000000,999.000000,21.000000,1757.000000,15.000000,166497.000000,2031.000000,19182.000000,7673.000000,60000.000000,60000.000000,60000.000000,25.000000,108101.000000,0.0,850.000000,525.000000,38.000000,1.000000,1.000000,21926.000000

In [52]:
print(df_games.iloc[21711,2])

animal world gather hula hoop championship goal hoop long possiblethis represent card goal rid card player drop quotdrop hoopquoteach player start card hand plus card openly turn choose play open card hand player play card demand player play quothigherquot quotlowerquot player follow quotdoublequot   play card player double break double neighboring card joker player play drop hoop roundwhen player leave player get rid card card play round player reach   point win game


In [4]:
df_na = df_games[df_games['Description'].isnull()]
print(df_na)
# Set description of game to its name
df_games.loc[14243, 'Description'] = 'Timeline: Sports et Loisirs'

        BGGId                         Name Description  YearPublished  \
14243  170984  Timeline: Sports et Loisirs         NaN           2014   

       GameWeight  AvgRating  BayesAvgRating   StdDev  MinPlayers  MaxPlayers  \
14243         1.0    6.64387         5.53658  1.62527           2           8   

       ...  Rank:partygames  Rank:childrensgames  Cat:Thematic Cat:Strategy  \
14243  ...            21926                21926             0            0   

       Cat:War  Cat:Family  Cat:CGS  Cat:Abstract  Cat:Party  Cat:Childrens  
14243        0           0        0             0          0              0  

[1 rows x 48 columns]


In [5]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21925 entries, 0 to 21924
Data columns (total 48 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   BGGId                21925 non-null  int64  
 1   Name                 21925 non-null  object 
 2   Description          21925 non-null  object 
 3   YearPublished        21925 non-null  int64  
 4   GameWeight           21925 non-null  float64
 5   AvgRating            21925 non-null  float64
 6   BayesAvgRating       21925 non-null  float64
 7   StdDev               21925 non-null  float64
 8   MinPlayers           21925 non-null  int64  
 9   MaxPlayers           21925 non-null  int64  
 10  ComAgeRec            16395 non-null  float64
 11  LanguageEase         16034 non-null  float64
 12  BestPlayers          21925 non-null  int64  
 13  GoodPlayers          21925 non-null  object 
 14  NumOwned             21925 non-null  int64  
 15  NumWant              21925 non-null 

In [6]:
# check if all BGGIds are unique
df_games['BGGId'] = df_games['BGGId'].astype('string')

df_games['BGGId'].describe()

count     21925
unique    21925
top           1
freq          1
Name: BGGId, dtype: object

In [7]:
#for features with categorical data

df_games.describe(include=('object', 'bool'))

,Name,Description,GoodPlayers,Family,ImagePath
count,21925,21925,21925,6663,21908
unique,21521,21904,78,1456,21904
top,Robin Hood,dice throne game intriguing dice tactical card...,[],18xx,https://cf.geekdo-images.com/6vlkpViMExKmOvg0C...
freq,6,4,19946,103,2


In [8]:
df_games['ImagePath'][0]

'https://cf.geekdo-images.com/rpwCZAjYLD940NWwP3SRoA__original/img/yR0aoBVKNrAmmCuBeSzQnMflLYg=/0x0/filters:format(jpeg)/pic4718279.jpg'

In [9]:
#df_matrix = df_df_games.set_index('Name', inplace=True)

In [10]:
df_games.head()

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
0,1,Die Macher,die macher game seven sequential political rac...,1986,4.3206,7.61428,7.10363,1.57979,3,5,...,21926,21926,0,1,0,0,0,0,0,0
1,2,Dragonmaster,dragonmaster tricktaking card game base old ga...,1981,1.9630,6.64537,5.78447,1.45440,3,4,...,21926,21926,0,1,0,0,0,0,0,0
2,3,Samurai,samurai set medieval japan player compete gain...,1998,2.4859,7.45601,7.23994,1.18227,2,4,...,21926,21926,0,1,0,0,0,0,0,0
3,4,Tal der Könige,triangular box luxurious large block tal der k...,1992,2.6667,6.60006,5.67954,1.23129,2,4,...,21926,21926,0,0,0,0,0,0,0,0
4,5,Acquire,acquire player strategically invest business t...,1964,2.5031,7.33861,7.14189,1.33583,2,6,...,21926,21926,0,1,0,0,0,0,0,0


### While there may be duplicate names, the BGG identities are unique. As we may have version differences that or different games with the same name, we will use the BGGId as the identifier.

## -----------------------------------------------------------------------------------------





In [11]:
# get game descriptions as features
df_description = pd.DataFrame()
df_description[['BGGId', 'Name', 'Description']] = df_games[['BGGId', 'Name', 'Description']]
df_description.reset_index(inplace=True)
df_description.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21925 entries, 0 to 21924
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   index        21925 non-null  int64 
 1   BGGId        21925 non-null  string
 2   Name         21925 non-null  object
 3   Description  21925 non-null  object
dtypes: int64(1), object(2), string(1)
memory usage: 685.3+ KB


# Create Feature Matrix Using Count Vectorizer

In [12]:
cv = CountVectorizer()

count_matrix = cv.fit_transform(df_description["Description"])
print("Count Matrix:", count_matrix.toarray())

Count Matrix: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [13]:
print(cv.get_feature_names())

['aa', 'aaa', 'aaagame', 'aaaunitslike', 'aachen', 'aacute', 'aacutebraacutezoljameghalt', 'aacutefrica', 'aacutefricaplus', 'aacutegil', 'aacuteguila', 'aacutehiacutetott', 'aacuteldozata', 'aacutelloacute', 'aacuteltal', 'aacutelvare', 'aacuterbol', 'aacuterea', 'aacutereas', 'aacuterpaacute', 'aacuterpaacuted', 'aacuterpaacutedhaacutez', 'aacutesgard', 'aacutesgardrock', 'aacutetvaacuteltoztathatjaacutek', 'aacutevido', 'aafter', 'aagewwer', 'aagun', 'aalst', 'aamir', 'aampa', 'aampampt', 'aampwp', 'aan', 'aang', 'aapep', 'aar', 'aare', 'aarghtect', 'aarin', 'aarklash', 'aaron', 'aaronit', 'aarre', 'ab', 'aba', 'ababa', 'abaci', 'abacus', 'abacuskismet', 'abacusspiele', 'abacusspieleadmittedly', 'abacusspielemore', 'abadan', 'abaddon', 'abaddonat', 'abajo', 'abalone', 'abande', 'abandon', 'abandonada', 'abandonadas', 'abandonado', 'abandonar', 'abandonase', 'abandonedthe', 'abandonsthe', 'abans', 'abascal', 'abate', 'abba', 'abbas', 'abbasid', 'abbess', 'abbeville', 'abbey', 'abbeyf

C:\Users\tdkla\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
#df_matrix = pd.DataFrame.from_records(data=count_matrix.toarray(), columns=cv.get_feature_names())

In [15]:
df_matrix_2 = pd.DataFrame()
df_matrix_2[['BGGId','Name']]=df_games[['BGGId','Name']]
df_matrix_2.set_index('Name', inplace=True)

In [16]:
count_matrix.shape

(21925, 127348)

## Create Similarity Matrix

In [17]:
similarity_matrix = cosine_similarity(count_matrix)

In [18]:
similarity_matrix.shape

(21925, 21925)

In [34]:
def game_recommender(game_title):
    # get index for game title
    index = np.where(df_matrix_2.index==game_title)[0][0]
    print(index)
    # get scores for the game title, sort, and get first 5 highest scoring game titles
    scores = list(enumerate(similarity_matrix[index]))
    print(scores[0:5])
    recommendations = sorted(scores, key=lambda x:x[1], reverse=True)[0:9]
    
    data_list = []
    for i in recommendations:
        inner_list=[]
        df_temp = df_games[df_games['Name']==df_matrix_2.index[i[0]]]
        inner_list.extend(df_temp.drop_duplicates('Name')['Name'])
        inner_list.extend(df_temp.drop_duplicates('Name')['ImagePath'])
        inner_list.extend(df_temp.drop_duplicates('Name')['BayesAvgRating'])
        inner_list.extend(df_temp.drop_duplicates('Name')['Rank:boardgame'])
        inner_list.extend(df_temp.drop_duplicates('Name')['MinPlayers'])
        inner_list.extend(df_temp.drop_duplicates('Name')['MaxPlayers'])
        data_list.append(inner_list)
        
    return data_list

In [48]:
game_recommender('Hula-Hoo!')


21711
[(0, 0.1526996982067525), (1, 0.5061488305827644), (2, 0.2769774944155841), (3, 0.16425344333030073), (4, 0.08092259783398602)]


[['Hula-Hoo!',
  'https://cf.geekdo-images.com/f9egoir9Utiiyy8JGg0xqA__original/img/H-Qsq3p9n3B9sazK_NqjSDGje-Y=/0x0/filters:format(jpeg)/pic5908887.jpg',
  5.52973,
  12914,
  2,
  6],
 ['Larry',
  'https://cf.geekdo-images.com/mO8r6oQ3XDd-EvGUzOK0nA__original/img/yRhBSMxfhF_L37-pFL_thssZCKA=/0x0/filters:format(png)/pic1886728.png',
  5.48215,
  19844,
  2,
  8],
 ['Bad Bunnies',
  'https://cf.geekdo-images.com/d_GSPnJ8Sdvl9DIHI1tObQ__original/img/dUZcl5anrGLvJl6oXH9xxTJG9D0=/0x0/filters:format(jpeg)/pic2365458.jpg',
  5.48945,
  19186,
  2,
  6],
 ['Gelb gewinnt!',
  'https://cf.geekdo-images.com/vb2pJwkUD7wVM5PZ0uViRw__original/img/wusLM6roXr2yZpeWXhZmBg3975Y=/0x0/filters:format(jpeg)/pic185665.jpg',
  5.45804,
  20856,
  2,
  8],
 ['ReCURRRing',
  'https://cf.geekdo-images.com/RecHspyabsFtoyhjdSOJog__original/img/HDI6q6mEYvqiPL9zBg2TYfLjSNI=/0x0/filters:format(jpeg)/pic6134658.jpg',
  5.5553,
  9915,
  3,
  5],
 ['Escalation!',
  'https://cf.geekdo-images.com/tZFY4X7Z4DP9T1-e1VPzPg

In [22]:
import pickle
pickle.dump(similarity_matrix, open('similarity_scores_desc.pkl', 'wb'))